In [53]:
# 3.0 LLMs and Chat Models (08:07)

from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

# 3.1 Predict Messages (05:04)

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

In [ ]:
#3.2 Prompt Templates (07:59)

template = PromptTemplate.from_template(
    "What is the distance between {country_a} and {country_b}",
)

prompt = template.format(country_a="Korea", country_b="Mexico")

chat.predict(prompt)

In [ ]:
#3.2 Prompt Templates (07:59)

template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a geography expert. And you only reply in {language}."),
        ("ai", "안녕, 나는 {name}이야."),
        (
            "human",
            "What is the distance between {country_a} and {country_b}. Also, what is your name?",
        ),
    ]
)

prompt = template.format_messages(
    language="Korean",
    name="세종",
    country_a="Chosun",
    country_b="Rome",
)

chat.predict_messages(prompt)

In [ ]:
#3.3 OutputParser and LCEL (11:51)

from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items = text.strip().split(",")
        return list(map(str.strip, items))
    
# p = CommaOutputParser()

# p.parse("Hello, how, are, you")

In [ ]:
#3.3 OutputParser and LCEL (11:51)

template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a list generating machine. Everything you are asked will be answered with a comma separated list of max {max_items} in lowercase. Do NOT reply with anything else.",
        ),
        ("human", "{question}"),
    ]
)

# prompt = template.format_messages(max_items=10, question="What are the colors?")

# result = chat.predict_messages(prompt)

# p = CommaOutputParser()

# p.parse(result.content)

In [13]:
#3.3 OutputParser and LCEL (11:51)

chain = template | chat | CommaOutputParser()

chain.invoke(
    {
        "max_items": 5,
        "question": "What are the pokemons?",
    },
)

['pikachu', 'charmander', 'bulbasaur', 'squirtle', 'jigglypuff']

In [59]:
# 3.4 Chaining Chains (14:11)

chef_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a world-class international chef. You create easy to follow recipies for any type of cuisine with easy to find ingredients.",
        ),
        ("human", "I want to cook {cuisine} food."),
    ]
)

chef_chain = chef_prompt | chat

translate_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a world class translator. You translate messages into {language} so that elementary school students can understand them.",
        ),
        ("human", "{messages}"),
    ]
)

translate_chain = translate_prompt | chat


In [62]:
# 3.4 Chaining Chains (14:11)

veg_chef_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a vegetarian chef specializ ed on making traditional recipes vegetarian. You find alternative ingredients and explain their preparation. You don't radically modify the recipe. If there is no alternative for a food just say you don't know how to recipe it.",
        ),
        ("human", "{recipe}"),
    ]
)

veg_chain = veg_chef_prompt | chat

final_chain = {"recipe": chef_chain} | veg_chain

final_chain.invoke(
    {
        "cuisine": "indian",
    }
)

Great choice! Indian cuisine is known for its bold flavors and aromatic spices. Let's start with a classic and easy recipe for Chicken Tikka Masala. Here's what you'll need:

Ingredients:
- 1 lb boneless, skinless chicken breasts, cut into bite-sized pieces
- 1 cup plain yogurt
- 2 tbsp lemon juice
- 2 tbsp vegetable oil
- 1 large onion, finely chopped
- 3 cloves garlic, minced
- 1-inch piece of ginger, grated
- 1 can (14 oz) crushed tomatoes
- 1 tbsp garam masala
- 1 tsp ground cumin
- 1 tsp ground coriander
- 1 tsp paprika
- 1/2 tsp turmeric
- 1/2 tsp cayenne pepper (adjust to taste)
- Salt and pepper to taste
- Fresh cilantro, chopped for garnish
- Cooked rice or naan bread for serving

Instructions:
1. In a bowl, combine the yogurt, lemon juice, 1 tbsp vegetable oil, 1 tbsp garam masala, 1 tsp ground cumin, 1 tsp ground coriander, 1 tsp paprika, 1/2 tsp turmeric, 1/2 tsp cayenne pepper, salt, and pepper. Add the chicken pieces and mix well to coat. Marinate for at least 1 hour, or 

AIMessageChunk(content="To make a vegetarian version of Chicken Tikka Masala, we can substitute the chicken with a plant-based alternative such as tofu or paneer. Here's how you can adapt the recipe:\n\nIngredients:\n- 1 lb firm tofu or paneer, cut into bite-sized pieces\n- 1 cup plain yogurt (you can use dairy-free yogurt for a vegan version)\n- 2 tbsp lemon juice\n- 2 tbsp vegetable oil\n- 1 large onion, finely chopped\n- 3 cloves garlic, minced\n- 1-inch piece of ginger, grated\n- 1 can (14 oz) crushed tomatoes\n- 1 tbsp garam masala\n- 1 tsp ground cumin\n- 1 tsp ground coriander\n- 1 tsp paprika\n- 1/2 tsp turmeric\n- 1/2 tsp cayenne pepper (adjust to taste)\n- Salt and pepper to taste\n- Fresh cilantro, chopped for garnish\n- Cooked rice or naan bread for serving\n\nInstructions:\n1. Instead of marinating chicken, marinate the tofu or paneer in the yogurt, lemon juice, 1 tbsp vegetable oil, spices, salt, and pepper for at least 1 hour, or overnight for best results.\n\n2. Follow 